In [1]:
import base64
import json
import pandas as pd
import numpy as np
import requests
import tensorflow as tf

In [2]:
df = pd.read_csv("data/Cleaned_HR_Data_Analysis.csv")

In [3]:
from pprint import PrettyPrinter
pp = PrettyPrinter()
pp.pprint(requests.get("https://wahyuazizi-mlops.up.railway.app/v1/models/employee-performance-model"))

<Response [200]>


In [4]:
CATEGORICAL_FEATURES = {
    "EmployeeStatus": 2,
    "EmployeeType": 3,
    "PayZone": 3,
    "EmployeeClassificationType": 3,
    "DepartmentType": 6,
    "GenderCode": 2,
    "RaceDesc": 5,
    "MaritalDesc": 4,
    "TrainingType": 2,
    "TrainingOutcome": 4
}

NUMERICAL_FEATURES = [
    "CurrentEmployeeRating",
    "EngagementScore",
    "SatisfactionScore",
    "Work-LifeBalanceScore",
    "TrainingDurationDays",
    "TrainingCost",
    "Age"
]
selected_features = list(CATEGORICAL_FEATURES.keys()) + NUMERICAL_FEATURES
inputs = df.loc[53, selected_features].to_dict()
inputs


{'EmployeeStatus': 'Active',
 'EmployeeType': 'Contract',
 'PayZone': 'Zone B',
 'EmployeeClassificationType': 'Full-Time',
 'DepartmentType': 'Sales',
 'GenderCode': 'Male',
 'RaceDesc': 'White',
 'MaritalDesc': 'Widowed',
 'TrainingType': 'Internal',
 'TrainingOutcome': 'Incomplete',
 'CurrentEmployeeRating': 5,
 'EngagementScore': 3,
 'SatisfactionScore': 4,
 'Work-LifeBalanceScore': 5,
 'TrainingDurationDays': 3,
 'TrainingCost': 442.56,
 'Age': 48}

In [5]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples":{
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://wahyuazizi-mlops.up.railway.app/v1/models/employee-performance-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
print(prediction)

None


In [9]:
import json
import base64
import requests
import tensorflow as tf

# Memilih fitur yang valid dari dataset
# selected_features = [col for col in list(CATEGORICAL_FEATURES.keys()) + NUMERICAL_FEATURES if col in df.columns]
selected_features = list(CATEGORICAL_FEATURES.keys()) + NUMERICAL_FEATURES

# Ambil data dari baris ke-53
inputs = df.loc[53, selected_features].to_dict()

def prepare_json(inputs: dict):
    """
    Mengonversi dictionary menjadi format JSON untuk request ke TensorFlow Serving
    """
    feature_spec = {}

    for key, value in inputs.items():
        if value is None:  # Jika ada nilai None, ubah jadi nilai default (misalnya 0 atau "")
            value = 0 if isinstance(value, (int, float)) else ""

        if isinstance(value, float):
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, int):
            feature_spec[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, str):
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

    # Serialisasi contoh data ke format TensorFlow Serving
    example = tf.train.Example(features=tf.train.Features(feature=feature_spec)).SerializeToString()
    encoded_example = base64.b64encode(example).decode()

    # Format JSON yang sesuai untuk TensorFlow Serving
    payload = {
        "signature_name": "serving_default",
        "instances": [{"b64": encoded_example}]
    }

    return json.dumps(payload)

# Membuat JSON dari input data
json_data = prepare_json(inputs)

# Endpoint model di Railway
endpoint = "https://wahyuazizi-mlops.up.railway.app/v1/models/employee-performance-model:predict"

# Mengirim request ke API
headers = {"Content-Type": "application/json"}
response = requests.post(endpoint, data=json_data, headers=headers)

# Menampilkan hasil prediksi
if response.status_code == 200:
    prediction = response.json().get("predictions", None)
    print("Prediction:", prediction)
else:
    print(f"Error {response.status_code}: {response.text}")


Error 400: {
    "error": "Name: <unknown>, Feature: BusinessUnit (data type: string) is required but could not be found.\n\t [[{{function_node __inference_serve_tf_examples_fn_927287}}{{node ParseExample/ParseExampleV2}}]]"
}


In [ ]:
"""
0 (PIP) → Kinerja buruk, butuh perbaikan segera.
1 (Fully Meets) → Memenuhi harapan pekerjaan dengan baik.
2 (Exceeded) → Melebihi ekspektasi dalam kinerja.
3 (Need Improvement) → Performa kurang baik, tapi belum sampai tahap PIP.
"""